In [1]:
import matplotlib.pyplot as plt 
import pickle
import networkx as nx
import re
import os
import json

In [23]:
# load graph object from file
G = pickle.load(open('G_politicans.pickle', 'rb'))

In [24]:
G.nodes(data=True)

NodeDataView({'A.T._Frank': {}, 'A._B._Swindell': {}, 'A._Breeze_Harper': {}, 'A._E._Henning': {}, 'A._F._Vandeventer': {}, 'A._Holly_Patterson': {}, 'A._Mitchell_Palmer': {}, 'A._P._Tugwell': {}, 'A._Perry_Gordy': {}, 'A._R._McCabe': {}, 'A._Ronald_Button': {}, 'A._Shane_Massey': {}, 'Aaron_Bean': {}, 'Aaron_Bernstine': {}, 'Aaron_Chatterji': {}, 'Aaron_Clausen': {}, 'Aaron_Coleman': {}, 'Aaron_Crossley': {}, 'Aaron_D._Ford': {}, 'Aaron_Freeman_(politician)': {}, 'Aaron_Frey': {}, 'Aaron_Griesheimer': {}, 'Aaron_Kaufer': {}, 'Aaron_Kaufman_(politician)': {}, 'Aaron_Libby': {}, 'Aaron_Lieberman': {}, 'Aaron_Ling_Johanson': {}, 'Aaron_M._Dana': {}, 'Aaron_McMullen': {}, 'Aaron_McWilliams': {}, 'Aaron_Michlewitz': {}, 'Aaron_Miller_(politician)': {}, 'Aaron_Ortiz': {}, 'Aaron_Peskin': {}, 'Aaron_Peterson': {}, 'Aaron_Pilkington': {}, 'Aaron_Regunberg': {}, 'Aaron_Rouse': {}, 'Aaron_S._Watkins': {}, 'Aaron_Schock': {}, 'Aaron_T._Bliss': {}, 'Aaron_Vega': {}, 'Aaron_von_Ehlinger': {}, 'Aar

In [26]:
from tqdm import tqdm

wiki_pages = "C:/Users/Mathilde/Documents/DTU/E24/SocialGraf/wiki_pages"

for node in tqdm(list(G.nodes()), desc="Processing nodes"):
    node_file = f"{node}.json"
    if os.path.exists(os.path.join(wiki_pages, node_file)):
        with open(os.path.join(wiki_pages, node_file), 'r', encoding="utf-8") as f:
            try:
                data = json.load(f)
                wikitext = data["query"]["pages"]
                page = list(wikitext.keys())[0]
                wikitext = wikitext[page]["revisions"][0]["*"]
                match = re.search(r'\bparty\b.*?\|([^]]+)\]\]', wikitext, re.IGNORECASE)
                if match:
                    party = match.group(1)
                    if len(party) < 50:
                        G.nodes[node]['party'] = party
            except:
                print(f"Error in file: {node_file}")
                continue

Processing nodes:   0%|          | 0/20534 [00:00<?, ?it/s]

Processing nodes: 100%|██████████| 20534/20534 [00:30<00:00, 667.62it/s]


In [46]:
for node in G.nodes():
    if 'party' in G.nodes[node] and (G.nodes[node]['party'] in ['Democratic', 'Democrat', 'Democratic Party', 'Democratic Party (United States)', 'DFL', 'Democratic (DFL)', 'Democratic-Farmer-Labor', 'Democratic–Farmer–Labor', 'Democratic-Farmer-Labor Party', 'Democratic-Farmer-Labor Party (Minnesota)', 'DFL party'] or re.search(r'democrat', G.nodes[node]['party'], re.IGNORECASE)):
        G.nodes[node]['party'] = 'Democratic'

    if 'party' in G.nodes[node] and (G.nodes[node]['party'] in ['Republican', 'Republican Party', 'Republican Party (United States)', 'GOP', 'Texas Republican Party'] or re.search(r'republican', G.nodes[node]['party'], re.IGNORECASE)):
        G.nodes[node]['party'] = 'Republican'

    if 'party' in G.nodes[node] and re.search(r'green', G.nodes[node]['party'], re.IGNORECASE):
        G.nodes[node]['party'] = 'Green'

    if 'party' in G.nodes[node] and re.search(r'libertarian', G.nodes[node]['party'], re.IGNORECASE):
        G.nodes[node]['party'] = 'Libertarian'

    if 'party' in G.nodes[node] and re.search(r'independent', G.nodes[node]['party'], re.IGNORECASE):
        G.nodes[node]['party'] = 'Independent'

    if 'party' in G.nodes[node] and re.search(r'socialist', G.nodes[node]['party'], re.IGNORECASE):
        G.nodes[node]['party'] = 'Socialist'
  

In [47]:
parties = ['Democratic', 'Republican', 'Libertarian', 'Green', 'Independent', 'Constitution', 'Socialist', 'Conservative']

count_party = 0
no_party = 0
for node in G.nodes():
    if 'party' in G.nodes[node] and G.nodes[node]['party'] not in parties:
        print(f"Node {node} has a party {G.nodes[node]['party']} not in the list of parties.")
        count_party += 1
    if 'party' not in G.nodes[node]:
        no_party += 1
print(f"Total nodes with parties not in the list: {count_party}")
print(f"Total nodes with no party: {no_party}")

Node Aaron_S._Watkins has a party Prohibition not in the list of parties.
Node Adina_Sash has a party district leader not in the list of parties.
Node Alan_Cranston has a party Senate Majority Whip not in the list of parties.
Node Alan_Simpson_(American_politician) has a party Senate Minority Whip not in the list of parties.
Node Albert_S._Porter has a party 1952 not in the list of parties.
Node Alejandro_Cruz_(politician) has a party New Progressive Party not in the list of parties.
Node Alfred_Hayes_Jr. has a party Progressive Party and the Bull Moose initiative not in the list of parties.
Node Alice_Green has a party Al "Grandpa" Lewis not in the list of parties.
Node Alice_Hausman has a party Saint Paul not in the list of parties.
Node Alison_Collins has a party John Burton not in the list of parties.
Node Amherst_B._Cheney has a party Prohibition not in the list of parties.
Node Amy_Kremer has a party 2016 presidential election not in the list of parties.
Node Andrea_Kieffer has a

In [48]:
from collections import Counter

party_counts = Counter(nx.get_node_attributes(G, 'party').values())
print(party_counts)

Counter({'Democratic': 9688, 'Republican': 9278, 'Independent': 135, 'Socialist': 41, 'Libertarian': 39, 'Green': 31, 'Prohibition': 18, 'Progressive': 14, 'Saint Paul': 8, 'Senate Majority Leader': 8, 'Senate Minority Whip': 7, 'Constitution': 6, 'Vermont Progressive': 5, 'House Minority Leader': 5, 'Reform': 4, 'Progressive Party': 4, 'Nonpartisan': 4, 'House Majority Leader': 4, 'Minneapolis': 4, 'Senate Majority Whip': 3, 'Bloomington': 3, "People's Party": 3, 'Anoka County': 3, 'Black Panther': 3, 'Dakota': 3, 'Le Sueur': 3, 'House Minority Whip': 3, 'Hennepin County': 3, 'Working Families': 3, 'Peace and Freedom': 3, 'third-party candidate': 3, 'Dakota County': 3, 'House Committee on the Chinese Communist Party': 3, 'Constitution Party': 3, 'House Majority Whip': 3, 'Chippewa': 2, 'Communist Party': 2, 'Anoka': 2, 'Citizens Party': 2, 'R': 2, 'Populist': 2, 'Forward Party': 2, 'Brooklyn Center': 2, 'Benton': 2, 'Kittson': 2, 'Forward': 2, 'United States President': 2, 'Farmer-Lab

In [49]:

nodes_to_remove = [node for node in G.nodes() if 'party' not in G.nodes[node] or G.nodes[node]['party'] not in parties]
G.remove_nodes_from(nodes_to_remove)

In [51]:
len(G.nodes(data=True))

19219

In [52]:
# save graph object to file
pickle.dump(G, open('G_politicans_w_party.pickle', 'wb'))